In [6]:
import os
import fitz
import spacy
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

import argparse

pd.set_option('display.max_colwidth', None)

ZMIANY W FUNKCJACH UTILS_DATA_CLEANING.py

In [7]:
project_dir = os.getcwd()
input_path = "/Users/LILI/Projects_studia/projekt_grupowy/3_nlp_2/nlp-group2/data"

# utils_dataprep.py - bez zmian

In [9]:
#branch 12
import os
import fitz

def read_documents(input_path):
    """
    Read text content from the first four pages of each PDF document in the specified folder.

    Args:
    - input_path (str): The path to the folder containing the PDF documents.

    Returns:
    - documents (list): A list of lists, where each inner list contains the filename
      and "CHCPL" - the combined text content of the first four pages of the corresponding PDF document.
    """
    documents = []
    if os.path.isdir(input_path):
        for filename in sorted(os.listdir(input_path)):  # Sort to ensure consistent order
            file_path = os.path.join(input_path, filename)
            if filename.endswith('.pdf'):
                try:
                    with fitz.open(file_path) as pdf:
                        combined_text = ""
                        for page_number in range(min(4, pdf.page_count)):
                            try:
                                page = pdf.load_page(page_number)
                                footer = 80  
                                page_text = page.get_text("text", clip=(0, 0, page.rect.width, page.rect.height - footer))
                                combined_text += page_text + " "
                            except Exception as e:
                                pass 
                        combined_text = combined_text.replace("\n", "")
                        documents.append([filename, combined_text])
                except Exception as e:
                    pass 
    else:
        print("Invalid input path. Please provide a valid folder path.")
    return documents


In [10]:
def load_to_pd(input_path):
    """
    Load text data from PDF documents in the specified folder into a pandas DataFrame.

    Args:
    - input_folder (str): The path to the folder containing the PDF documents.

    Returns:
    - text_df (DataFrame): A pandas DataFrame containing the text data from the PDF documents.
      The DataFrame has two columns: "filename" and "CHPCL". Each row corresponds to a PDF document,
      where "filename" contains the name of the PDF file, and "CHPCL" contains the combined text content
      of the first four pages of the PDF.

    """
    text = read_documents(input_path)
    text_df = pd.DataFrame(text)
    current_columns = text_df.columns
    new_columns = [f"column{i+1}" for i in range(len(current_columns))]
    text_df.columns = new_columns
    text_df.rename(columns={"column1": "filename", "column2":"CHPCL"}, inplace=True)
    return text_df

In [11]:
text_df = load_to_pd(input_path)

In [12]:
text_df.head()

filename  \
0          Charakterystyka-10025-2021-12-22-8574_N-2022-01-05.pdf   
1         Charakterystyka-10062-2023-01-11-11269_A-2023-01-22.pdf   
2  Charakterystyka-10088-20200721000000-6861_N-20200916000402.pdf   
3         Charakterystyka-10174-2023-04-12-13484_A-2023-04-29.pdf   
4         Charakterystyka-10252-2022-06-17-10632_A-2022-06-24.pdf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [7]:
text_df.iloc[865:866 ,:]

filename  \
865  Charakterystyka-47283-2023-10-31-15084_D-2023-11-10.pdf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

# utils_info_extract.py

In [13]:
import fitz
import re
import spacy
import os
import pandas as pd

def apply_replacements(df, column):
    """
    Apply replacements to the specified column in the DataFrame.

    Args:
    - df (DataFrame): The DataFrame.
    - column (str): The name of the column to apply replacements to.
    - replacements (dict): A dictionary where keys are old values to be replaced
      and values are new values.

    Returns:
    - df (DataFrame): The DataFrame with replacements applied to the specified column.
    """
    replacements = {
        "NAZWA WŁASNA": "NAZWA",
        "LECZNICZNEGO": "LECZNICZEGO ",
        "LECZNICZEGO": "LECZNICZEGO ",
        "PODUKTU": "PRODUKTU",
        "PRODUKU": "PRODUKTU",
        "PODUKTU": "PRODUKTU",
        "1.NAZWA ": "1. NAZWA ",
        "1.NAZWA ": "1. NAZWA ",
        "1 nazwa" :"1. NAZWA ",
        "SKLAD": "SKŁAD",
        "2. skład" : "2. SKŁAD ",
        "2.SKŁAD ": "2. SKŁAD ",
        "2 SKŁAD" :"2. SKŁAD ",
        "2 skład": "2. SKŁAD ",
        "4.1 Wskazania": "4.1. Wskazania",
        "4.1Wskazania": "4.1. Wskazania",
        "4.1.Wskazania": "4.1. Wskazania",
        "2.0\\4-": "",
        "\\":"",
        "/":"",
        "jest wskazany": "wskazania"        
    }
    for old_value, new_value in replacements.items():
        df[column] = df[column].str.upper().str.replace(old_value, new_value).str.lower()
    return df


def extract_columns(df, column):
    df = apply_replacements(df, column) 
    for index, row in df.iterrows():
        #nazwa
        start_index_nazwa = row[column].find("1")
        end_index_nazwa = row[column].find("2.") if row[column].find("2.") != -1 else row[column].find("2")
        nazwa = row[column][start_index_nazwa:end_index_nazwa].strip()
        df.at[index, 'nazwa'] = nazwa    
        #sklad
        start_index_sklad = row[column].find("2.") if row[column].find("2.") != -1 else row[column].find("2 skład ilościowy i jakościowy ")
        end_index_sklad = row[column].find("3.") if row[column].find("3.") != -1 else row[column].find("4.1")
        sklad = row[column][start_index_sklad:end_index_sklad].strip()
        df.at[index, 'sklad'] = sklad
        #wskazania
        start_index_wskazania = row[column].find("4.1") if row[column].find("4.1") != -1 else row[column].find("wskazania") 
        end_index_wskazania = row[column].find("4.2")
        wskazania = row[column][start_index_wskazania:end_index_wskazania].strip()
        df.at[index, 'wskazania'] = wskazania
        
    return df[['filename', 'nazwa', 'sklad', 'wskazania']]


In [14]:
text_extracted = extract_columns(text_df, 'CHPCL') 

In [ ]:
 text_df = load_to_pd(input_path)
    text_extracted = extract_columns(text_df, 'CHPCL') 
    text_cleaned= process_text_columns(text_extracted)

In [15]:
text_extracted.iloc[865:866,:]

,filename,nazwa,sklad,wskazania
865,Charakterystyka-47283-2023-10-31-15084_D-2023-11-10.pdf,"1. nazwa produktu leczniczego ebozan, 2,5 mg, tabletki ebozan, 5 mg, tabletki ebozan, 10 mg, tabletki ebozan 20 mg, tabletki","2. skład jakościowy i ilościowy ebozan, 2,5 mg, tabletki każda tabletka zawiera 2,5 mg torasemidu (torasemidum). substancja pomocnicza o znanym działaniu: każda tabletka zawiera 48 mg laktozy (w postaci laktozy jednowodnej). ebozan, 5 mg, tabletki każda tabletka zawiera 5 mg torasemidu (torasemidum). substancja pomocnicza o znanym działaniu: każda tabletka zawiera 97 mg laktozy (w postaci laktozy jednowodnej). ebozan, 10 mg, tabletki każda tabletka zawiera 10 mg torasemidu (torasemidum). substancja pomocnicza o znanym działaniu: każda tabletka zawiera 193 mg laktozy (w postaci laktozy jednowodnej). ebozan, 20 mg, tabletki każda tabletka zawiera 20 mg torasemidu (torasemidum). substancja pomocnicza o znanym działaniu: każda tabletka zawiera 386 mg laktozy (w postaci laktozy jednowodnej). pełny wykaz substancji pomocniczych, patrz punkt 6.1.","4.1 wskazania do stosowania pochodzenia nerkowego. ponadto produkt leczniczy ebozan 2,5 mg i 5 mg, tabletki są wskazane w leczeniu nadciśnienia tętniczego pierwotnego. produkt leczniczy ebozan jest wskazany u dorosłych i młodzieży w wieku powyżej 12 lat."


In [67]:
#test wyszukiwania po podziale na kolumnuy - 128 wierszy z nadcisnieniem

#wyszukiwanie = 'Charakterystyka-36343-20170830000000-3286_A-20170921000413.pdf'
wyszukiwanie = ['Charakterystyka-38481-20190517000000-6797_B-20190616000045.pdf', 'Charakterystyka-33025-2023-11-09-15802_A-2023-11-14.pdf']
kolumna = 'filename'
#filtered_df = text_extracted[text_extracted[kolumna].str.contains(wyszukiwanie)]
filtered_df = text_extracted[text_extracted[kolumna].str.contains('|'.join(wyszukiwanie))]
filtered_df


,filename,nazwa,sklad,wskazania
431,Charakterystyka-33025-2023-11-09-15802_A-2023-11-14.pdf,"1. nazwa produktu leczniczego combogesic, 500 mg + 150 mg, tabletki powlekane","2. skład jakościowy i ilościowy każda tabletka zawiera 500 mg paracetamolu i 150 mg ibuprofenu. substancja pomocnicza o znanym działaniu: laktoza jednowodna, 3,81 mg. pełny wykaz substancji pomocniczych, patrz punkt 6.1.","4.1 wskazania do stosowania doraźne leczenie bólu głowy, bólu migrenowego, bólu pleców, bólów miesiączkowych, bólu zębów, bólów mięśni, bólów związanych z objawami przeziębienia i grypy, bólu gardła i gorączki."
602,Charakterystyka-38481-20190517000000-6797_B-20190616000045.pdf,"1. nazwa produktu leczniczego enplerasa, 25 mg, tabletki powlekane enplerasa, 50 mg, tabletki powlekane","2. skład jakościowy i ilościowy 25 mg: każda tabletka powlekana zawiera 25 mg eplerenonu. 50 mg: każda tabletka powlekana zawiera 50 mg eplerenonu. substancja pomocnicza o znanym działaniu: laktoza jednowodna 25 mg: każda tabletka powlekana zawiera 34,5 mg laktozy jednowodnej. 50 mg: każda tabletka powlekana zawiera 69 mg laktozy jednowodnej. pełny wykaz substancji pomocniczych, patrz punkt 6.1.","4.1 wskazania do stosowania eplerenon jest wskazany jako: - terapia dodana do standardowego leczenia obejmującego stosowanie beta-adrenolityków, w celu zmniejszenia ryzyka umieralności i zachorowalności z przyczyn sercowo-naczyniowych u pacjentów w stanie stabilnym, z zaburzeniem czynności lewej komory serca (lvef ≤ 40 %, ang. left ventricular ejection fraction − lvef) oraz klinicznymi objawami niewydolności serca po przebytym niedawno zawale serca. - terapia dodana do standardowego leczenia, w celu zmniejszenia ryzyka umieralności i zachorowalności z przyczyn sercowo-naczyniowych u dorosłych pacjentów z (przewlekłą) niewydolnością serca (klasa ii wg nyha) oraz z zaburzeniami czynności lewej komory serca (lvef ≤30%) (patrz punkt 5.1)."


In [11]:
print(f"Znaleziono {filtered_df.shape[0]} dla wyszukiwania: {wyszukiwanie}.")

NameError: name 'filtered_df' is not defined

# utils_data_cleaning.py - zmiany

In [13]:
import spacy
import re
import pandas as pd
from datetime import datetime

nlp = spacy.load("pl_core_news_lg")


def clean_formatting(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.replace('\n', ' ').strip().lower()  # Normalize text (convert to lowercase)
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'\d+', '', text)  # Remove digits
    return text

def remove_stop_words(text, specified_words_to_remove):
    words = text.split()
    filtered_words = [word for word in words if word not in specified_words_to_remove]
    return ' '.join(filtered_words)

def remove_duplicated_words(text):
    words = text.split()
    unique_words = list(dict.fromkeys(words))  # Maintain order and remove duplicates
    return ' '.join(unique_words)


def lemmatize_text(text):
    doc = nlp(text)  
    lemmatized_text = " ".join(token.lemma_ for token in doc if token.pos_ == "NOUN" or token.pos_ == "PROPN")  
    return lemmatized_text

def process_text_columns(df):
    columns_to_process = ['nazwa', 'sklad', 'wskazania']
    stop_words_nazwa = ['nazwa', 'produktu', 'leczniczego', 'mg']
    stop_words_sklad = ['skład', 'jakościowy', 'ilościowy',  "pełny", "wykaz",  "substancji", \
                         "pomocniczych", "patrz", "punkt", "substancje", "pomocnicze", "mg", "znanym", "każdy", "każda"]
    stop_words_wskazania = [ "wskazania", " do ", "stosowania", "zapobieganie", "stosowania", "wskazany", "wskazanie",\
                            "leczniczy", "leczenie", "leczeniu", " i ", "i ", " e " " o ", " u ", " z ", " przy ", " od ", " do ", " na ", " w ", " oraz ", " kiedy ", " który ", " która ", " które ", " których ",  "pacjent", "patrz", "punkty", "osób", "produkt", "substancja", "pomocniczy", "pomocnicza" , \
                                "metody", "celu", "mieszaną"]
    
    for column in columns_to_process:
        if column == 'nazwa':
            df.loc[:, column] = df[column].apply(clean_formatting)
            df.loc[:, column] = df[column].apply(remove_stop_words, specified_words_to_remove=stop_words_nazwa)
        #    df.loc[:, column] = df[column].apply(lemmatize_text)
            df.loc[:, column] = df[column].apply(remove_duplicated_words)


        elif column == 'sklad':
            df.loc[:, column] = df[column].apply(clean_formatting)
            df.loc[:, column] = df[column].apply(remove_stop_words, specified_words_to_remove=stop_words_sklad)
            df.loc[:, column] = df[column].apply(lemmatize_text)
            df.loc[:, column] = df[column].apply(remove_duplicated_words)
            
        
        elif column == 'wskazania':
            df.loc[:, column] = df[column].apply(clean_formatting)
            df.loc[:, column] = df[column].apply(remove_stop_words, specified_words_to_remove=stop_words_wskazania)
            df.loc[:, column] = df[column].apply(lemmatize_text)
            df.loc[:, column] = df[column].apply(remove_duplicated_words)
    
    return df



In [14]:
text_cleaned= process_text_columns(text_extracted)


In [15]:
text_cleaned.iloc[695:696,:]

,filename,nazwa,sklad,wskazania
695,Charakterystyka-42003-2023-11-17-16860_B-2024-02-21.pdf,mantreda tabletki powlekane,i tabletka powlekać zawierać rywaroksaban substancja pomocniczy o działać laktoza jednowodny,do profilaktyka żylny choroba zakrzepowozatorowy żchzza u dorosły pacjent po przebyty planowy alloplastyka staw biodrowy lub kolanowy zakrzepica żyć głęboki zżg i zatorowość płucny zp oraz nawrotowy punkt z hemodynamiczn niestabilny


In [68]:
#test wyszukiwania po podziale na kolumnuy - 128 wierszy z nadcisnieniem

#wyszukiwanie = 'Charakterystyka-36343-20170830000000-3286_A-20170921000413.pdf'
wyszukiwanie = ['Charakterystyka-38481-20190517000000-6797_B-20190616000045.pdf', 'Charakterystyka-33025-2023-11-09-15802_A-2023-11-14.pdf']
kolumna = 'filename'
#filtered_df = text_extracted[text_extracted[kolumna].str.contains(wyszukiwanie)]
filtered_clean = text_cleaned[text_cleaned[kolumna].str.contains('|'.join(wyszukiwanie))]
filtered_clean


,filename,nazwa,sklad,wskazania
431,Charakterystyka-33025-2023-11-09-15802_A-2023-11-14.pdf,combogesic tabletki powlekane,i tabletka zawierać paracetamol ibuprofen substancja pomocniczy o działać laktoza jednowodny,do doraźny ból głowa migrenowy pleców miesiączkowy zębów mięśni związać z objaw przeziębienie i grypa gardło gorączka
602,Charakterystyka-38481-20190517000000-6797_B-20190616000045.pdf,enplerasa tabletki powlekane,i tabletka powlekać zawierać eplerenon substancja pomocniczy o działać laktoza jednowodny,do eplerenon być jako terapia dodać standardowy leczenie obejmować stosować betaadrenolityk w zmniejszyć ryzyko umieralność i zachorowalność z przyczyna sercowonaczyniowy u pacjent stan stabilny zaburzenie czynność lewy komora serce lvef ang left ventricular ejection fraction oraz kliniczny objaw niewydolność po przebytym niedawno zawał leczyć dorosły przewlekła klasa ii według nyha punkt


In [22]:
def convert_to_dict(df):
    list_of_docs = []  
    for index, row in df.iterrows():
        doc = {
            "title": row["nazwa"],  
            "text": row["sklad"] + " " + row["wskazania"],  
            "filename": row["filename"],  
            "timestamp": datetime.now()  
        }
        list_of_docs.append(doc)
    return list_of_docs

In [23]:
list_of_docs = convert_to_dict(text_cleaned)

# utils_search_engine.py

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def prepare_tfidf_representation(documents):
    corpus = [doc['text'] for doc in documents]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)
    return vectorizer, tfidf_matrix

In [25]:
# BRANCH 11
def search_product_by_indication(wskazania_pattern, products, vectorizer, tfidf_matrix):
    if not products:
        return []
    lemmatized_wskazania = lemmatize_text(wskazania_pattern)
    query_tfidf = vectorizer.transform([lemmatized_wskazania])
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix)
    scores = list(enumerate(cosine_similarities[0]))
    rounded_scores = [(index, round(score, 3)) for index, score in scores]
    sorted_rounded_scores = sorted(rounded_scores, key=lambda x: x[1], reverse=True)
    matching_products = [{'product_name': products[idx]['title'], 'filename': products[idx]['filename'], 'score': score} for idx, score in sorted_rounded_scores]
    return matching_products


In [26]:
tfidf_vectorizer, tfidf_matrix = prepare_tfidf_representation(list_of_docs)

In [27]:
q_input = 'cukrzyca'

In [28]:
def capitalize_first_letter(text):
    """
    Capitalize the first letter of the text.
    If the text is empty, return it as is.
    """
    if text:
        return text[0].upper() + text[1:]
    else:
        return text

wskazania_pattern = (q_input) 
matching_products = search_product_by_indication(wskazania_pattern, list_of_docs, tfidf_vectorizer, tfidf_matrix)
threashold = 0.0
filtered_products = [product for product in matching_products if product['score'] > threashold]
for product in filtered_products:
    
    product_name_capitalized = capitalize_first_letter(product["product_name"])
    print(f'{product["filename"]},"{product_name_capitalized}",{product["score"]:.3f}')

Charakterystyka-43642-2022-09-23-12032_B-2022-10-05.pdf,"Telmisartan aurovitas tabletki",0.198
Charakterystyka-43643-2022-09-23-12032_B-2022-10-05.pdf,"Telmisartan aurovitas tabletki",0.198
Charakterystyka-38066-20210108000000-9169_B-20210123000830.pdf,"Telmisartan orion tabletki",0.193
Charakterystyka-46194-2023-06-26-12511_D-2023-07-14.pdf,"Lisinopril grindeks tabletki",0.184
Charakterystyka-44797-2022-06-30-11620_B-2022-07-08.pdf,"Telmisartan medical valley tabletki",0.178
Charakterystyka-41664-2022-01-27-1748_D-2022-02-21.pdf,"Lisinopril aurovitas tabletki",0.176
Charakterystyka-9408-2021-11-18-6230_N-2021-11-23.pdf,"Prinivil tabletki",0.155
Charakterystyka-20171-20210204000000-7328_N-20210218001743.pdf,"Captopril jelfa tabletki",0.138
Charakterystyka-45664-2023-06-19-14384_B-2023-07-18.pdf,"Dabigatran etexilate pharma kapsułki twarde",0.131
Charakterystyka-8069-20200930000000-8568_A-20201014000922.pdf,"Venoruton forte tabletki",0.127
Charakterystyka-43822-2023-08-04-15331_B-2023-1

In [29]:
print(f"Number of outputs found: {len(filtered_products)}")

Number of outputs found: 26


## utils_search_similar.py

1. uzytkownik podaje nazwe do nowego pliku lub sciezke do niego
2. plik jest wczytany funkcja read_documents
3. plik jest poddany podzialowi na kolumny i oczyszczaniu
4. plik jest zlematyzowany 
5. plik jest porownany z pozostalymi wskazaniami w list_docs jesli chodzi o sklad i wskazanie, nie nazwe
6. lista produktow jest sformatowane tak jak dla query

In [30]:
new_folder_path = "/Users/LILI/Projects_studia/projekt_grupowy/3_nlp_2/test_file_search"
new_file = load_to_pd(new_folder_path)
new_file_extracted = extract_columns(new_file, 'CHPCL') 
new_file_cleaned = process_text_columns(new_file_extracted)

def convert_to_dict_new_file(df):
    list_of_docs = []  
    for index, row in df.iterrows():
        doc = {
            "filename": row["filename"], 
            "nazwa": row["nazwa"],  
            "sklad": row["sklad"],
            "wskazania":  row["wskazania"]
        }
        list_of_docs.append(doc)
    return list_of_docs
new_file_list_of_docs = convert_to_dict_new_file(new_file_extracted)
new_file_sklad = [doc['sklad'] for doc in new_file_list_of_docs]
new_file_wskazania = [doc['wskazania'] for doc in new_file_list_of_docs]

In [31]:
text_df = load_to_pd(input_path)
text_extracted = extract_columns(text_df, 'CHPCL') 
text_cleaned= process_text_columns(text_extracted)
text_dict= convert_to_dict_new_file(text_cleaned)
text_dict_sklad = [doc['sklad'] for doc in text_dict]
text_dict_wskazania = [doc['wskazania'] for doc in text_dict]

In [43]:
new_file_list_of_docs

[{'filename': 'Charakterystyka-2767-2021-06-23-9423_A-2021-08-18.pdf',
  'nazwa': 'gutron tabletki',
  'sklad': 'i tabletka zawierać chlorowodorek midodryny midodrini hydrochloridum',
  'wskazania': 'do ciężki niedociśnienie ortostatyczny spowodować zaburzenie czynność autonomiczny układ nerwowy gdy nie mieć możliwość leczyć przyczynowy'}]

In [44]:
model = SentenceTransformer('LaBSE')
list_of_docs_new = convert_to_dict_new_file(text_cleaned) #przygotowanie slownika z czterema columnami ze wszystkich istniejacych CHPCL

doc_embeddings = model.encode([doc['sklad'] for doc in list_of_docs_new])

index = faiss.IndexFlatL2(doc_embeddings[0].shape[0])  # Initialize the index
index.add(np.array(doc_embeddings)) 

while True:
    #query = input("Enter the file name to list similar products (top 5 outputs): ")
    query ="/Users/LILI/Projects_studia/projekt_grupowy/3_nlp_2/test_file_search"
    #new_folder_path = "/Users/LILI/Projects_studia/projekt_grupowy/3_nlp_2/test_file_search"
    
    #new_file = load_to_pd(args.file)
    new_file = load_to_pd(query)
    new_file_extracted = extract_columns(new_file, 'CHPCL')  
    new_file_cleaned = process_text_columns(new_file_extracted) 
    new_file_list_of_docs = convert_to_dict_new_file(new_file_extracted) 
    #new_file_sklad = [doc['sklad'] for doc in new_file_list_of_docs]
    #new_file_wskazania = [doc['wskazania'] for doc in new_file_list_of_docs]


    #query_embedding = model.encode([query])[0]
    query_embedding = model.encode([new_file_list_of_docs])[0]
    _, indices = index.search(np.array([query_embedding]), 5)

    #top_documents = [(args.file[idx], 1.0) for idx in indices[0]]
    #top_documents = [(new_file_list_of_docs.file[idx], 1.0) for idx in indices[0]]
    top_documents = [(new_file_list_of_docs[idx], 1.0) for idx in indices[0]]

    for i, (doc, similarity) in enumerate(top_documents, start=1):
        print(f"Document {i}:")
        print(f'{doc["filename"]},"{doc["nazwa"]}",{similarity:.3f},{doc["sklad"]}')

IndexError: list index out of range

In [132]:
#PREVIOUS VERSION
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

model = SentenceTransformer('LaBSE')
doc_embeddings = model.encode([doc['text'] for doc in list_of_docs])
index = faiss.IndexFlatL2(doc_embeddings[0].shape[0])  # Initialize the index
index.add(np.array(doc_embeddings)) 
while True:
        query = input("Enter the file name to list similar products (top 5 outputs): ")
        query_embedding = model.encode([query])[0]
        _, indices = index.search(np.array([query_embedding]), 5)

        top_documents = [(list_of_docs[idx], 1.0) for idx in indices[0]]

        for i, (doc, similarity) in enumerate(top_documents, start=1):
            print(f"Document {i}:")
            print(f"Similarity Score: {similarity}")
            print(f"Title: {doc['title']}")
            print(f"Text: {doc['text']}")
            print(f"Filename: {doc['filename']}")
            print()

IndexError: list index out of range

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

def run_similarity_model(list_of_docs):

    model = SentenceTransformer('LaBSE')
    doc_embeddings = model.encode([doc['text'] for doc in list_of_docs])

    index = faiss.IndexFlatL2(doc_embeddings[0].shape[0])  # Initialize the index
    index.add(np.array(doc_embeddings)) 

    while True:
        query = input("Enter the file name to list similar products (top 5 outputs): ")
        query_embedding = model.encode([query])[0]
        _, indices = index.search(np.array([query_embedding]), 5)

        top_documents = [(list_of_docs[idx], 1.0) for idx in indices[0]]

        for i, (doc, similarity) in enumerate(top_documents, start=1):
            print(f"Document {i}:")
            print(f"Similarity Score: {similarity}")
            print(f"Title: {doc['title']}")
            print(f"Text: {doc['text']}")
            print(f"Filename: {doc['filename']}")
            print()

# Check versions


In [45]:
import pandas as pd 
pd.__version__

'2.2.1'

In [46]:
import subprocess

packages = [
    "nltk==3.8.1",
    "PyMuPDF==1.24.2",
    "scikit-learn==1.4.1.post1",
    "sentence-transformers==2.5.1",
    "spacy==3.7.4",
    "torch==2.2.1",
    "transformers==4.38.2",
    "faiss-cpu==1.8.0",
    "pandas==2.2.1"
]

for package in packages:
    package_name = package.split('==')[0]
    try:
        result = subprocess.run(["pip", "show", package_name], capture_output=True, text=True)
        if result.returncode == 0:
            version_info = [line for line in result.stdout.split('\n') if "Version" in line]
            if version_info:
                print(f"{package_name} is installed: {version_info[0]}")
            else:
                print(f"{package_name} is installed but version information is not available.")
        else:
            print(f"{package_name} is not installed.")
    except Exception as e:
        print(f"An error occurred while checking {package_name}: {str(e)}")


nltk is installed: Version: 3.8.1
PyMuPDF is installed: Version: 1.24.2
scikit-learn is installed: Version: 1.4.1.post1
sentence-transformers is installed: Version: 2.5.1
spacy is installed: Version: 3.7.4
torch is installed: Version: 2.2.2
transformers is installed: Version: 4.38.2
faiss-cpu is installed: Version: 1.8.0
pandas is installed: Version: 2.2.1
